# OSI Lat Long
### This Notebook uses the XY of the point featureclass and adds that to each related row in a related table

In [1]:
import arcpy
import numpy
import pandas as pd

In [2]:
gdb = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb"
svcPntSource = r"C:\GISData\Data\Snapshot\mxElectric.geodatabase\Electric\main.eServicePoint"
#custAcctSource = r"C:\GISData\Data\Snapshot\mxElectric.geodatabase\main.eCUSTOMERACCOUNT"
svcPntDest = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\eServicePoint"
custAcctDest = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\eCUSTOMERACCOUNT"
svcPntDestFlds = ["OID@","POINT_X","POINT_Y","PHASEDESIGNATION"]
custAcctDestFlds = ["OID@","POINT_X","POINT_Y","PHASEDESIGNATION","SERVICEPOINTOBJECTID"]
custAcctOutFlds = ["OID@","SERVICEPOINTOBJECTID","INSTALL_NUM","POINT_X","POINT_Y","PHASEDESIGNATION"]
custAnalysis = ["SERVICEPOINTOBJECTID","INSTALL_NUM","POINT_X","POINT_Y","AVGCONSUMPTION","MAXCONSUMPTION"]
custAcctFile = r'C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\GIS.csv'

OSI Phase Designation
1 – A
2 – B
3 – C
12 – AB
13 – AC
23 – BC
123 – ABC

In [3]:
def getPhaseDesignation(phaseDesignation):
        if phaseDesignation is None:
                ph = 0
                return ph
        if phaseDesignation == 1:
                ph = 3
                return ph
        if phaseDesignation == 2:
                ph = 2
                return ph
        if phaseDesignation == 3:
                ph = 23
                return ph
        if phaseDesignation == 4:
                ph = 1
                return ph
        if phaseDesignation == 5:
                ph = 13
                return ph
        if phaseDesignation == 6:
                ph = 12
                return ph
        if phaseDesignation == 7:
                ph = 123
                return ph

In [4]:
arcpy.env.workspace = gdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4326)

In [5]:
arcpy.env.workspace

'C:\\Users\\friendde\\Documents\\ArcGIS\\Projects\\OSILatLong\\OSILatLong.gdb'

In [6]:
if arcpy.Exists(svcPntDest):
    print(f'Found {svcPntDest}')
    arcpy.Delete_management (svcPntDest)
print(f'Copying {svcPntSource} to {svcPntDest}')
arcpy.Copy_management(svcPntSource,svcPntDest)
if arcpy.Exists(svcPntDest):
    print(f'Copy Success, adding XY')
    arcpy.AddXY_management(svcPntDest)
    print(f'XY completed')

Found C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\eServicePoint
Copying C:\GISData\Data\Snapshot\mxElectric.geodatabase\Electric\main.eServicePoint to C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\eServicePoint
Copy Success, adding XY
XY completed


In [7]:
if arcpy.Exists(custAcctDest):
    print(f'Copy Success')
    arcpy.AddField_management(custAcctDest, "POINT_X", "DOUBLE")
    arcpy.AddField_management(custAcctDest, "POINT_Y", "DOUBLE")
    arcpy.AddField_management(custAcctDest, "PHASEDESIGNATION", "LONG")
else:
    print(f'Copy Failed!')

Copy Success


In [8]:
edit = arcpy.da.Editor(gdb)
edit.startEditing(False, False)
edit.startOperation()

In [9]:
with arcpy.da.SearchCursor(svcPntDest,svcPntDestFlds) as svcpnts:
        for svcpnt in svcpnts:
                whereClause = "SERVICEPOINTOBJECTID = {}".format(svcpnt[0])
                #print(whereClause,svcpnt[1])
                with arcpy.da.UpdateCursor(custAcctDest,custAcctDestFlds,whereClause) as uc:
                    #print(row)
                    for row in uc:
                            row[1] = svcpnt[1]
                            row[2] = svcpnt[2]
                            row[3] = getPhaseDesignation(svcpnt[3])
                            uc.updateRow(row)

In [10]:
edit.stopOperation()
edit.stopEditing(True)

http://pro.arcgis.com/en/pro-app/arcpy/get-started/working-with-numpy-in-arcgis.htm

In [11]:
nparr = arcpy.da.TableToNumPyArray(custAcctDest,custAcctOutFlds,skip_nulls=True)
#nparr = arcpy.da.TableToNumPyArray(custAcctDest,custAcctOutFlds,null_value=-9999)
pdarr = pd.DataFrame(nparr)
pdarr.to_csv(custAcctFile,header=False, index=False)

\\gruadmin.gru.com\fs\Groups\OMS Replacement Project\Documents for OSII\Customer and Premise Files

### Use pandas to analyze customer consumption

Use arcpy [```TableToNumPyArray()```](http://pro.arcgis.com/en/pro-app/arcpy/data-access/tabletonumpyarray.htm)

In [32]:
nparr = arcpy.da.TableToNumPyArray(custAcctDest,["SERVICEPOINTOBJECTID","POINT_X","POINT_Y","AVGCONSUMPTION","MAXCONSUMPTION"],skip_nulls=True)

In [33]:
df = pd.DataFrame(nparr)
df.head(5)

,SERVICEPOINTOBJECTID,POINT_X,POINT_Y,AVGCONSUMPTION,MAXCONSUMPTION
0,5,-82.255216,29.660429,997.0,1721.0
1,6,-82.256019,29.659257,1045.0,1728.0
2,7,-82.253436,29.659716,689.0,2117.0
3,9,-82.243694,29.702255,530.0,820.0
4,11,-82.253930,29.676644,1095.0,2435.0


[```gis.features.SpatialDataFrame()```](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html?highlight=spatialdataframe#arcgis.features.SpatialDataFrame.from_xy)

In [20]:
from arcgis.features import SpatialDataFrame
from arcgis.gis import GIS
from getpass import getpass
from IPython.display import display

In [17]:
sdf = SpatialDataFrame.from_xy(df,"POINT_X","POINT_Y")
gis = GIS(arcpy.GetActivePortalURL(), username=input("Enter User Name "), password=(getpass()))
portalDesc = arcpy.GetPortalDescription()
# search and list all items owned by connected user
#query=f'owner:{portalDesc["user"]["username"]} AND title:CW BaseMap'
#itemType="Feature Layer"
#sortField="title"
#sortOrder="asc"
# default max__items is 10
#maxItems=100
#m = gis.content.search(query,itemType,sortField,sortOrder,maxItems)

Enter User Name arcgiswebservices
········


In [31]:
m = gis.map('Gainesville,FL')
m
consumptionLyr = gis.content.import_data(sdf)
m.add_layer(consumptionLyr,options={"renderer":"GraduatedSymbolsRenderer","classificationField":"MAXCONSUMPtION"})

In [28]:
m

MapView(basemaps=['dark-gray', 'dark-gray-vector', 'gray', 'gray-vector', 'hybrid', 'national-geographic', 'oc…